# Dev: Display Project Progress

## Preamble

## Workflow

In [1]:
from tuttle.model import ICloudAccount

2022-01-20 22:04:09,358 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-20 22:04:09,359 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-01-20 22:04:09,359 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 22:04:09,360 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2022-01-20 22:04:09,360 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 22:04:09,360 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("icloudaccount")
2022-01-20 22:04:09,360 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 22:04:09,361 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("bank")
2022-01-20 22:04:09,361 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 22:04:09,361 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("bankaccount")
2022-01-20 22:04:09,362 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-20 22:04:09,362 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("contact")
2022-01-20 22:04:09,362 INFO sqlalchemy.engine.Engine [raw sql

In [2]:
my_icloud_account = ICloudAccount(
    user_name=input("icloud user name: ")
)

icloud user name:  mail@clstaudt.me


In [3]:
from tuttle.cloud import login_iCloud

In [4]:
icloud = login_iCloud(my_icloud_account)

iCloud password for account mail@clstaudt.me:  ·······


In [5]:
from tuttle.timetracking import calendar_to_timetracking_table
from tuttle.calendar import CloudCalendar

In [6]:
cal = CloudCalendar(
    icloud=icloud,
    name="TTTT",
)

/Users/cls/miniforge3/envs/tuttle/lib/python3.9/site-packages/pyicloud/services/calendar.py:81: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  "usertz": get_localzone().zone,


In [7]:
from pandera.typing import DataFrame

In [8]:
from tuttle.schema import TimeTracking

In [9]:
time_tracking: DataFrame[TimeTracking] = calendar_to_timetracking_table(cal)

/Users/cls/miniforge3/envs/tuttle/lib/python3.9/site-packages/pyicloud/services/calendar.py:53: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  "usertz": get_localzone().zone,


In [10]:
time_tracking

,begin,end,title,duration,tag
438,2022-01-17 08:00:00,2022-01-17 20:00:00,#EvadeCentralServices,0 days 12:00:00,#EvadeCentralServices
441,2022-01-19 17:00:00,2022-01-19 19:00:00,#EvadeCentralServices,0 days 02:00:00,#EvadeCentralServices
442,2022-01-18 08:00:00,2022-01-18 18:00:00,#HeatingRepair,0 days 10:00:00,#HeatingRepair
443,2022-01-19 08:15:00,2022-01-19 18:15:00,#HeatingRepair,0 days 10:00:00,#HeatingRepair
444,2022-01-18 12:00:00,2022-01-18 14:00:00,#EvadeCentralServices,0 days 02:00:00,#EvadeCentralServices
447,2022-01-20 08:15:00,2022-01-20 18:15:00,#HeatingRepair,0 days 10:00:00,#HeatingRepair
449,2022-01-20 12:00:00,2022-01-20 14:00:00,#HeatingRepair,0 days 02:00:00,#HeatingRepair


In [11]:
from tuttle.model import Project, Contract

In [12]:
import datetime

In [13]:
from tuttle.time import TimeUnit

In [14]:
contract_hr = Contract(
    title="Heating Repair Contract",
    start_date=datetime.date(2022,1,1),
    volume=128,
    unit=TimeUnit.hour,
)

In [15]:
proj_hr = Project(
    title="Heating Repair",
    tag="#HeatingRepair",
    start_date=datetime.date(2022,1,1),
    contract=contract_hr
)

In [32]:
def progress(
    project: Project,
    time_tracking_data: DataFrame[TimeTracking],
):
    tag = project.tag
    total_time = (
        time_tracking_data
        .filter(["tag", "duration"])
        .query(f"tag == @tag")
        .groupby("tag")
        .sum()
    )
    budget = project.contract.volume * datetime.timedelta(hours=1)
    return total_time.loc[tag]["duration"] / budget
    

In [33]:
progress(
    project=proj_hr,
    time_tracking_data=time_tracking
)

0.25